In [2]:
from src.utilities_rejection import RejectionReasonLabeling, label_reasons, label_reasons_column
from src.data_local import MergedDataPreprocessing
from src.lstm_encoder import LSTMEmbedding
from src.model_train import encode_label
from cashe import get_cashed_input
import numpy as np
import time

In [3]:
df_original = get_cashed_input()

df_original = df_original[['VISIT_ID','CREATION_DATE','NOTES','SERVICE_DESCRIPTION', 'SERVICE_TYPE', 'OASIS_IOS_DESCRIPTION','PROVIDER_DEPARTMENT','ICD10','Chief_Complaint','OUTCOME']]

In [4]:
preprocessing = MergedDataPreprocessing(df_original)
preprocessing.fast_icd_encode()

df_train, df_test = preprocessing.train_test_split_time()

In [5]:
reasons = ['MN-1-1']

train_rej = RejectionReasonLabeling(df_train)
train_rej.recoginze_label()
train_rej = RejectionReasonLabeling(df_test)
train_rej.recoginze_label()

df_train, df_test = label_reasons(df_train,df_test,reasons)

In [7]:
train_columns = list(df_train.columns)[3:]

X_train = df_train[train_columns[:-3]];  y_train = df_train[train_columns[-3:]]
X_test = df_test[train_columns[:-3]];  y_test = df_test[train_columns[-3:]]

y_train['OUTCOME'] = encode_label(y_train.OUTCOME.tolist()); y_test['OUTCOME'] = encode_label(y_test.OUTCOME.tolist())

C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_14584\2327790411.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_train['OUTCOME'] = encode_label(y_train.OUTCOME.tolist()); y_test['OUTCOME'] = encode_label(y_test.OUTCOME.tolist())


In [60]:
y_train

,OUTCOME,NPHIES_CODE,NPHIES_LABEL
242103,1,None,0
242104,0,CV-1-1,0
242107,0,CV-1-1,0
242108,0,CV-1-1,0
242109,0,CV-1-1,0
...,...,...,...
365593,1,None,0
365588,0,None,0
367850,0,MN-1-1,2
365597,1,None,0


In [8]:
lstm_embedding = LSTMEmbedding()
X_train_processed = lstm_embedding.preprocess_data(X_train)

In [9]:
outcomes_train = label_reasons_column(y_train);  outcomes_test  = label_reasons_column(y_test)

y_train = y_train.copy();  y_test = y_test.copy()

In [10]:
y_train = [1 if elem == 2 else 0 for elem in outcomes_train] ## keeping the medical reason
y_test  = [1 if elem == 2 else 0 for elem in outcomes_test]

In [ ]:
print('Training Model')
lstm_embedding.train_model(X_train_processed, y_train,epochs=20,batch_size=64)

print('Evaluating Model')
accuracy = lstm_embedding.evaluate_model(X_test, y_test)

print('Model Accuracy on test data: ',accuracy)

In [ ]:
lstm_embedding.input_shape

## Test Results

In [69]:
X_test_vectorized = lstm_embedding.embedding_vector(X_test[:3],reload_model=False)
X_test_vectorized

array([[ 214,  633,  975,  765,  566, 1556, 1443, 6169],
       [ 157,  296,  140,  162,  163,  164,  666, 1244],
       [ 276, 1075,  140,  162,  163,  164,  666, 1244]])

In [14]:
start_time = time.time()
lstm_encoder = LSTMEmbedding()
lstm_encoder.load_model()
X_test_vectorized = lstm_embedding.embedding_vector(X_test[:10],reload_model=True)

end_time = time.time()
execution_time = end_time - start_time
print("Execution time: {:.2f} seconds".format(execution_time))

Execution time: 0.13 seconds


In [15]:
X_test_vectorized

array([[  7,   8,  72,  19,   7,   8,  19,  12,   5, 424, 424, 530, 788,
          0,   0,   0],
       [316,  33, 208,  69, 316,  33, 208,  69,   0,   0,   0,   0,   0,
          0,   0,   0],
       [  5, 119,   5, 119,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [422,   5, 674, 607, 422,   5, 674, 607,   0,   0,   0,   0,   0,
          0,   0,   0],
       [422,   5, 674, 607, 422,   5, 674, 607,   0,   0,   0,   0,   0,
          0,   0,   0],
       [ 77,  77,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [ 70,   5, 123, 300, 465, 556,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [307, 107, 307, 107,  29, 300, 465, 556,   0,   0,   0,   0,   0,
          0,   0,   0],
       [ 63, 421,  52,  63, 421,  52, 300, 465, 556,   0,   0,   0,   0,
          0,   0,   0],
       [264,  52, 187,   2, 264,  52, 187,   2, 300, 465, 556,   0,   0,
          0,   0,   0]])

In [16]:
def vectorize(COUNT_VALS):
      preds = lstm_encoder.embedding_vector(X_test[:COUNT_VALS],reload_model=True)
      return

def calculate_excecution():
  exs = []
  COUNT_VALS = 50
  for i in range(10):
    start_time = time.time()

    vectorize(COUNT_VALS)

    end_time = time.time()

    execution_time = end_time - start_time
    exs.append(execution_time)

  average = np.array(exs).mean()
  return average/COUNT_VALS

In [17]:
calculate_excecution()

0.001288477897644043

In [18]:
X_test

,SERVICE_DESCRIPTION,SERVICE_TYPE,OASIS_IOS_DESCRIPTION,PROVIDER_DEPARTMENT,ICD10,Chief_Complaint
365589,Automated complete Blood cell and differentia...,laboratory,Automated complete Blood cell count,ORTHOPAEDIC SERVICE - العظام,Adhesive capsulitis of shoulder,left shoulder pain no truma 1w over use during...
305745,FUCIDIN CREAM 2% 30 GM,medication-codes,FUCIDIN CREAM 2% 30 GM,DERMATOLOGY & VENEREOLOGY SERV - جلدية,Viral warts,None
305744,Cryotherapy of single skin lesion,procedures,CRYOTHERAPY OF SINGLE SKIN LESION,DERMATOLOGY & VENEREOLOGY SERV - جلدية,Viral warts,None
307086,"Dressing of wound, large",procedures,"Dressing of wound, large",DERMATOLOGY & VENEREOLOGY SERV - جلدية,Viral warts,None
307088,"Dressing of wound, large",procedures,"Dressing of wound, large",DERMATOLOGY & VENEREOLOGY SERV - جلدية,Viral warts,None
...,...,...,...,...,...,...
352722,CLARA 10MG TABLET,medication-codes,CLARA 10MG TABLET,EMERGENCY ROOM SERVICES - طوارى,Acute upper respiratory infections of multiple...,None
352721,AZI-ONCE,medication-codes,AZI-ONCE,EMERGENCY ROOM SERVICES - طوارى,Acute upper respiratory infections of multiple...,None
352720,KAFOSED SYRUP 15MG-5ML,medication-codes,KAFOSED SYRUP 15MG-5ML,EMERGENCY ROOM SERVICES - طوارى,Acute upper respiratory infections of multiple...,None
352719,Unlisted Code,services,INTRAVENOUS INJECTION(INJ.ROOM,EMERGENCY ROOM SERVICES - طوارى,Acute upper respiratory infections of multiple...,None
